## Imports

In [31]:

# Import packages
import pandas as pd
import numpy as np
import io

import scipy.stats as stats
import pingouin as pg

import random
from random import randrange

import matplotlib.pyplot as plt
import seaborn as sns

import argparse

import Orthoscripts

pd.options.mode.chained_assignment = None 

#### Import genelists

In [2]:
# Asterias rubens
Astrub = Orthoscripts.readBED("Data/Genelists/Asterias.rubens.genelist.bed")

# Holothuria leucospilota
Holleu = Orthoscripts.readBED("Data/Genelists/Holothuria.leucospilota.genelist.bed")

# Paracentrotus livides
Parliv = Orthoscripts.readBED("Data/Genelists/Paracentrotus.lividus.genelist.bed")

# Branchiostoma lanceolatum
Bralan = Orthoscripts.readBED("Data/Genelists/Branchiostoma.lanceolatum.genelist.bed")

# Branchiostoma floridae
Braflo = Orthoscripts.readBED("Data/Genelists/Branchiostoma.floridae.genelist.bed", 's')

# Marthasterias glacialis
Margla = Orthoscripts.readBED("Data/Genelists/Marthasterias.glacialis.genelist.bed")

# Pecten maximus
Pecmax = Orthoscripts.readBED("Data/Genelists/Pecmax.genelist.bed", 's')

# Stichopus chloronotus
Stichl = Orthoscripts.readBED("Data/Genelists/Stichopus.chloronotus.genelist.bed")

# Amphiura filiformis 
Ampfil = Orthoscripts.readBED("Data/Genelists/Amphiura.filiformis.genelist.bed")

# Ephydatia muelleri
Ephmue = Orthoscripts.readBED("Data/Genelists/Ephmue.genelist.bed", 's')

# Ancestor 
AniAnc = Orthoscripts.readBED("Data/Genelists/AniAnc.genelist.bed", 's')
BilAnc = Orthoscripts.readBED("Data/Genelists/BilAnc.genelist.bed", 's')

#### Import ortholog files

In [3]:
# Import orthologs
Astrub_Holleu = np.loadtxt("Data/Orthologs/Asterias.rubens+Holothuria.leucospilota.txt", dtype = "str")

Astrub_Parliv = np.loadtxt("Data/Orthologs/Asterias.rubens+Paracentrotus.lividus.txt", dtype = "str")

Holleu_Parliv = np.loadtxt("Data/Orthologs/Holothuria.leucospilota+Paracentrotus.lividus.txt", dtype = "str")

Margla_Bralan = np.loadtxt("Data/Orthologs/Marthasterias.glacialis+Branchiostoma.lanceolatum.txt", dtype = "str")

Margla_Pecmax = np.loadtxt("Data/Orthologs/Marthasterias.glacialis+Pecten.maximus.txt", dtype = "str")

Margla_Stichl = np.loadtxt("Data/Orthologs/Marthasterias.glacialis+Stichopus.chloronotus.txt", dtype = "str")

Pecmax_Bralan = np.loadtxt("Data/Orthologs/Pecten.maximus+Branchiostoma.lanceolatum.txt", dtype = "str")

Stichl_Bralan = np.loadtxt("Data/Orthologs/Stichopus.chloronotus+Branchiostoma.lanceolatum.txt", dtype = "str")

Stichl_Pecmax = np.loadtxt("Data/Orthologs/Stichopus.chloronotus+Pecten.maximus.txt", dtype = "str")

Pecmax_Holleu = np.loadtxt("Orthology pipeline/orthologs/Pecmax+Holleu_sensitive.txt", dtype = "str")

Holleu_Bralan = np.loadtxt("Orthology pipeline/orthologs/Holleu+Bralan_sensitive.txt", dtype = "str")

Pecmax_Bralan = np.loadtxt("Orthology pipeline/orthologs/Pecmax+Bralan_sensitive.txt", dtype = "str")

Pecmax_Braflo = np.loadtxt("Orthology pipeline/orthologs/Pecmax+Braflo_sensitive.txt", dtype = "str")

Holleu_Braflo = np.loadtxt("Orthology pipeline/orthologs/Holleu+Braflo_sensitive.txt", dtype = "str")

Holleu_Ampfil = np.loadtxt("Data/Orthologs/Holothuria.leucospilota+Amphiura.filiformis.txt", dtype = "str")

Braflo_Ephmue = np.loadtxt("Orthology pipeline/orthologs/Braflo+Ephmue_sensitive.txt", dtype = "str")

Holleu_Ephmue = np.loadtxt("Orthology pipeline/orthologs/Holleu+Ephmue_sensitive.txt", dtype = "str")

Pecmax_Ephmue = np.loadtxt("Orthology pipeline/orthologs/Pecmax+Ephmue_sensitive.txt", dtype = "str")

#### Sorting out the data

In [4]:
Astrub = Astrub.loc[Astrub['Chromosome'].str.contains('chr')]
Bralan = Bralan.loc[Bralan['Chromosome'].str.contains('BFL_')]
Braflo = Braflo.loc[Braflo['Chromosome'].str.contains('BFL_')]
Pecmax = Pecmax.loc[Pecmax['Chromosome'].str.contains('PYE_')]
Ephmue = Ephmue.loc[Ephmue['Chromosome'].str.contains('EMU_')]

# Ephmue genelist: remove suffix
Ephmue['Name'] = Ephmue['Name'].str.rsplit('.t1').str.get(0)

# Parliv genelist: select chromosomal scaffolds
Parliv = Orthoscripts.unscaff(Parliv, 100)
Ampfil = Orthoscripts.unscaff(Ampfil, 100)
Ephmue = Orthoscripts.unscaff(Ephmue, 600)

Astrub_Parliv = Orthoscripts.orthFix(Astrub_Parliv, 'B', 'Parliv_', 1)
Margla_Bralan = Orthoscripts.orthFix(Margla_Bralan, 'A', '.1', 0)
Margla_Stichl = Orthoscripts.orthFix(Margla_Stichl, 'A', '.1', 0)
Margla_Stichl = Orthoscripts.orthFix(Margla_Stichl, 'B', '.1', 0)
Margla_Pecmax = Orthoscripts.orthFix(Margla_Pecmax, 'B', '.1', 0)
Holleu_Ampfil = Orthoscripts.orthFix(Holleu_Ampfil, 'B', '.1', 0)
Holleu_Bralan = Orthoscripts.orthFix(Holleu_Bralan, 'B', '_', 0)

### 
-----

### Plots

In [ ]:
data = Orthoscripts.orthofind(Braflo, Ephmue, Braflo_Ephmue)
Orthoscripts.orthoplot(data, 'Amphioxus', 'Sponge', 'A', 'B')

------------------------------

### Test simulations

In [2]:
def simulator(Nchr = 20, Ngene = 100, Nevents = 10):

    # Make ancestor genome
    def makeancestor(Nchr, Ngene):
        ancestor = pd.DataFrame(columns = ['Chromosome'])
        for i in range(Nchr):
            row = {'Chromosome' : (i + 1)}
            for i in range(Ngene):
                    ancestor = pd.concat([ancestor, pd.DataFrame([row])], ignore_index = True)
        ancestor['Name'] = (ancestor.reset_index().index + 1)
        # ancestor['Name'] = 'g_' + ancestor['Name'].astype(str)

        return ancestor

    # Dummy BED files :: type 'anc' for ancestor, 'des' for descendant
    def dummyBED(genome, type):
        if type == 'anc':
            genome['Chromosome'] = 'AncChr' + genome['Chromosome'].astype(str)
            genome['Name'] = 'ancg_' + genome['Name'].astype(str)
            
        if type == 'des':
            genome['Chromosome'] = 'Chr' + genome['Chromosome'].astype(str)
            genome['Name'] = 'g_' + genome['Name'].astype(str)
        
        genome['Start'] = np.arange(len(genome))
        genome['End'] = np.arange(len(genome)) + 5
        
        genome = genome[['Chromosome', 'Start', 'End', 'Name']]
            
        return genome

    # Dummy ortholog file
    def dummyOrthologs(genome):
        orthologs = pd.DataFrame()
        
        orthologs['Orthologs'] = np.arange(len(genome)) + 1
        orthologs['speciesA'] = np.arange(len(genome)) + 1
        orthologs['speciesB'] = np.arange(len(genome)) + 1
        
        orthologs['Orthologs'] = 'orthologs_' + orthologs['Orthologs'].astype(str)
        orthologs['speciesA'] = 'ancg_' + orthologs['speciesA'].astype(str)
        orthologs['speciesB'] = 'g_' + orthologs['speciesB'].astype(str)
        
        orthologs = orthologs.to_numpy()
        
        return orthologs

    def mixing(genome, mixing):
        genes = genome['Name'].to_numpy()
        n = len(genes)
        for i in range(int(mixing * n)):
            g1, g2 = randrange(n), randrange(n)
            genes[g2], genes[g1] = genes[g1], genes[g2]

            genome['Name'] = genes
            # genome['Chromosome'] = f'{fuse1}x{fuse2}'
            
    def fusion(genome, chr, mixing = 0):
        '''
        inputs: 
        ancestor : 
            df with chromosome name | gene name
        mixing : 
            float between 0 and 1, where 1 implies 
            extreme mixing and 0 implies no mixing
        '''
        
        # Randomly select two chromosomes to fuse
        A = random.choice(chr)
        B = random.choice(chr)
        
        chr = [x for x in chr if x not in (A, B)]
        
        if A == B: # Just so the same chromosome isn't selected twice
            B = random.choice(chr)
            if A == B:
                B = random.choice(chr)

        fusion = ancestor.loc[ancestor['Chromosome'].isin([A, B])]
        
        # Apply mixing if required
        if mixing > 0:
            genes = fusion['Name'].to_numpy()
            n = len(genes)
            for i in range(int(mixing * n)):
                g1, g2 = randrange(n), randrange(n)
                genes[g2], genes[g1] = genes[g1], genes[g2]

            fusion['Name'] = genes
            fusion['Chromosome'] = f'{A}x{B}'
            log = f'FUS, AncChr{A} AncChr{B}, Chr{A}x{B}'
            
        else:
            fusion['Chromosome'] = f'{A}+{B}'
            log = f'FUS, AncChr{A} AncChr{B}, Chr{A}+{B}'
        
        # Remove the unfused chromosomes
        genome.drop(genome[genome['Chromosome'].isin([A, B])].index, inplace = True)
        genome = pd.concat([genome, fusion])
        
        return genome, log, chr

    def fission(genome, chr):
        # Randomly select a chromosome for fission
        A = random.choice(chr)
        fission = genome.loc[genome['Chromosome'] == A]
        chr.remove(A)
        
        pos = random.choice(range(1, Ngene))

        # Add the new chromosomes back into the genome
        chr1 = fission.iloc[: pos]
        chr1['Chromosome'] = f'{A}_1'
        
        chr2 = fission.iloc[pos :]
        chr2['Chromosome'] = f'{A}_2'
        
        # Remove the fission chromosome from the genome
        genome = pd.concat([genome, chr1, chr2])
        genome = genome[genome.Chromosome != A]
        
        log = f'FIS, AncChr{A}, Chr{A}_1 Chr{A}_2'
        
        return genome, log, chr

    def translocation(genome, chr):
        # Randomly select two chromosomes for translocation
        A = random.choice(chr)
        B = random.choice(chr)
        
        if A == B: # Just so the same chromosome isn't selected twice
            B = random.choice(chr)
        
        chr = [x for x in chr if x not in (A, B)]
        
        chrA = genome.loc[genome['Chromosome'] == A]
        chrB = genome.loc[genome['Chromosome'] == B]
        
        # Randomly select two break point positions
        posA = random.choice(range(1, Ngene))
        posB = random.choice(range(1, Ngene))
        
        r = np.random.uniform()
        if r <= 0.30:
            # Join the fragments to form recombinant chromosomes
            chr1 = pd.concat([chrA.iloc[: posA], chrB.iloc[posB :]])
            chr1['Chromosome'] = f'{A};{B}'
            chr2 = pd.concat([chrB.iloc[: posB], chrA.iloc[posA :]])
            chr2['Chromosome'] = f'{B};{A}'
            
            log = f'TR, AncChr{A} AncChr{B}, Chr{A};{B} Chr{B};{A}'
        
        else:
            chr1 = pd.concat([chrA.iloc[: posA]])
            chr1['Chromosome'] = f'{A};'
            chr2 = pd.concat([chrB, chrA.iloc[posA :]])
            chr2['Chromosome'] = f'{B};{A}'
        
            log = f'TR, AncChr{A} AncChr{B}, Chr{B};{A}'
            
        # Remove the original chromosomes from the genome
        genome = pd.concat([genome, chr1, chr2]).drop(genome[(genome['Chromosome'] == A) & (genome['Chromosome'] == B)].index)
        return genome, log, chr

    # Apply macro-rearrangements to the ancestor
    ancestor = makeancestor(Nchr, Ngene)
    chr = ancestor.Chromosome.unique().tolist()
    speciesA = ancestor.copy()

    events = []
    for event in range(Nevents):
        r = np.random.uniform()
        
        if r <= 0.30:
            if len(ancestor) < 2: continue
            speciesA, log, chr = fission(speciesA, chr)
            events.append(log)
        
        elif r <= 0.45:
            speciesA, log, chr = translocation(speciesA, chr)
            events.append(log)
        
        elif r <= 0.70:
            speciesA, log, chr = fusion(speciesA, chr)
            events.append(log)
        
        elif r <= 0.99:
            speciesA, log, chr = fusion(speciesA, chr, mixing = 0.5)
            events.append(log)
            
        else:
            # speciesA, log, chr = syntenyloss(speciesA, chr)
            # events.append(log)
            # print(log)
            continue
        
    ancestor = dummyBED(ancestor, 'anc')
    speciesA = dummyBED(speciesA, 'des')
    orthologs = dummyOrthologs(ancestor)
    
    return ancestor, speciesA, orthologs, events

In [46]:
simancestor, simspeciesA, simorthologs, simevents = simulator()
data = Orthoscripts.orthologies(simancestor, simspeciesA, simorthologs)

simevents = ' '.join(simevents)
simevents

'FUS, AncChr7 AncChr3, Chr7x3 FUS, AncChr15 AncChr12, Chr15x12 TR, AncChr1 AncChr4, Chr1;4 Chr4;1 FIS, AncChr5, Chr5_1 Chr5_2 FIS, AncChr17, Chr17_1 Chr17_2 FIS, AncChr10, Chr10_1 Chr10_2 FUS, AncChr6 AncChr11, Chr6+11 FIS, AncChr8, Chr8_1 Chr8_2 FUS, AncChr14 AncChr16, Chr14+16 FUS, AncChr20 AncChr19, Chr20+19'

In [44]:
df = pd.read_csv(io.StringIO(simevents), sep = ",")
df

,FUS,AncChr6 AncChr20,Chr6x20 FIS,AncChr4,Chr4_1 Chr4_2 FIS,AncChr2,Chr2_1 Chr2_2 FUS,AncChr1 AncChr18,Chr1x18 TR,AncChr12 AncChr15,...,AncChr7 AncChr10,Chr7x10 FUS,AncChr14 AncChr3,Chr14x3 FIS,AncChr9,Chr9_1 Chr9_2 FUS,AncChr19 AncChr17,Chr19+17 FUS,AncChr11 AncChr8,Chr11x8


In [30]:
c = simevents[1]
print(c)

TR, AncChr2 AncChr6, Chr6;2
